In [25]:
import backtrader as bt
import pandas as pd
import calendar
from datetime import datetime
import empyrical
import pyfolio as pf
import yfinance as yf
import warnings
warnings.filterwarnings('ignore')

In [26]:
# 定義齊全到期日的計算函數
def option_expiration(date):
    day = 21 - (calendar.weekday(date.year, date.month, 1) + 4) % 7
    return datetime(date.year, date.month, day)

In [27]:
# 定義回測策略
class SampleStrategy(bt.Strategy):
    def log(self, txt, dt=None, is_stock=True):
        # 根據是否為股票或期貨選擇不同的 datetime
        if is_stock:
            dt = dt or self.datas[0].datetime.datetime(0) # 使用 0050 的 datetime
        else:
            dt = dt or self.datas[1].datetime.datetime(0) # 使用 TXF 的 datetime
        print(f"{dt.isoformat()}, {txt}")

    def __init__(self):
        # 保存 0050 和 TXF 的資料
        self.dataclose_0050 = self.datas[0].close
        self.dataclose_txf = self.datas[1].close
        self.order_0050 = None
        self.order_txf = None
        self.first_trade_done = False
        self.cash_reserved_for_txf = 1000000

        # 用來追蹤 0050 的最高獲利
        self.highest_profit = 0
        self.stop_loss_triggered = False

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return
        if order.status in [order.Completed]:
            is_stock = order.data._name == "0050"
            if order.isbuy():
                self.log(f"""買入執行 {order.data._name},
                         價格: {order.executed.price:.2f},
                         成本: {order.executed.value:.2f},
                         手續費: {order.executed.comm:.2f}""", is_stock=is_stock)
            else:
                self.log(f"""賣出執行 {order.data._name},
                         價格: {order.executed.price:.2f},
                         成本: {order.executed.value:.2f},
                         手續費: {order.executed.comm:.2f}""", is_stock=is_stock)
                
            self.order_0050 = None
            self.order_txf = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return 
        is_stock = trade.data._name == "0050"
        self.log(f"操作利潤 {trade.data._name}, 淨利 {trade.pnl:.2f}", is_stock=is_stock)
    
    def next(self):
        position_size = self.getposition(data=self.datas[1]).size
        futures_date = self.datas[1].datetime.datetime(0)

        # 合約到期日檢查
        if option_expiration(futures_date).day == futures_date.day and futures_date.hour >= 13:
            if position_size != 0:
                self.close(data=self.datas[1])
                self.log("因到期日平倉 TXF 持倉", is_stock=False)
            return # 不再執行後續邏輯
        
        # 第一次交易：購買 0050 股票
        if not self.first_trade_done:
            available_cash_for_0050 = self.broker.getcash() - self.cash_reserved_for_txf
            size_0050 = int(available_cash_for_0050 / self.dataclose_0050[0])
            self.order_0050 = self.buy(data=self.datas[0], size=size_0050)
            self.log(f"創建 0050 買入訂單, 大小: {size_0050}", is_stock=True)
            self.first_trade_done = True
            return
        
        # 計算 0050 的持倉盈虧
        position_0050 = self.getposition(data=self.datas[0])
        current_profit = position_0050.size * (self.dataclose_0050[0] - position_0050.price)

        # 更新最高獲利
        if current_profit > self.highest_profit:
            self.highest_profit = current_profit

        # 獲利回撤 10% 時觸發做空 TXF
        if not self.stop_loss_triggered and current_profit < self.highest_profit * 0.9:
            self.log(f"0050 獲利回撤超過 10%, 創建 TXF 賣出訂單", is_stock=False)
            self.order_txf = self.sell(data=self.datas[1], size=2) # 賣出 2 口 TXF
            self.stop_loss_triggered = True

        # 當價格恢復時 (0050 盈利重新超過之前的 90%)，平倉 TXF
        if self.stop_loss_triggered and current_profit >= self.highest_profit * 0.9:
            self.log(f"0050 盈利回升, 平倉", is_stock=False)
            self.stop_loss_triggered = False
            

In [28]:
# 初始化 Cerebro
cerebro = bt.Cerebro()
# 使用 yfinance 加載 0050 的日 K 資料
data_0050 = yf.download("0050.TW", start="2019-03-04", end="2020-02-28").droplevel(
    "Ticker", axis=1
)
data_0050 = data_0050[["Open", "High", "Low", "Close", "Volume"]]
data_0050 = data_0050.reset_index()
data_0050["Date"] = pd.to_datetime(data_0050["Date"])

[*********************100%***********************]  1 of 1 completed


In [29]:
data_0050

Price,Date,Open,High,Low,Close,Volume
0,2019-03-04,15.923300,15.923300,15.748769,15.800101,33878280
1,2019-03-05,15.728234,15.789834,15.697435,15.759034,37601972
2,2019-03-06,15.779569,15.820635,15.759036,15.800101,78319168
3,2019-03-07,15.789835,15.820635,15.728235,15.738503,72959080
4,2019-03-08,15.646101,15.676901,15.563971,15.605036,39900800
...,...,...,...,...,...,...
236,2020-02-21,19.595775,19.659917,19.446106,19.510250,21604752
237,2020-02-24,19.296440,19.328512,19.178843,19.264368,54287744
238,2020-02-25,19.136081,19.349892,19.125389,19.317820,35360416
239,2020-02-26,19.061247,19.189533,19.007794,19.125389,60324012


In [30]:
data_feed_0050 = bt.feeds.PandasData(
    dataname=data_0050,
    name='0050',
    datetime=0,
    high=2,
    low=3,
    open=1,
    close=4,
    volume=5,
    plot=False
)

In [31]:
cerebro.adddata(data_feed_0050, name='0050')
cerebro.broker.setcommission(commission=0.001, name='0050')

In [32]:
# 加載 TXF 的 30 分鐘 K 線資料
df = pd.read_csv('TXF_30.csv')
df = df.dropna()
df['Date'] = pd.to_datetime(df['Date'])
df.index = df['Date']
df = df.between_time('08:45', '13:45')

In [33]:
df

,Date,Open,High,Low,Close,Volume
Date,,,,,,
2019-03-04 09:15:00,2019-03-04 09:15:00,10390.0,10400.0,10308.0,10309.0,28947
2019-03-04 09:45:00,2019-03-04 09:45:00,10309.0,10343.0,10294.0,10315.0,27199
2019-03-04 10:15:00,2019-03-04 10:15:00,10314.0,10325.0,10301.0,10313.0,11852
2019-03-04 10:45:00,2019-03-04 10:45:00,10313.0,10324.0,10276.0,10280.0,15081
2019-03-04 11:15:00,2019-03-04 11:15:00,10279.0,10293.0,10279.0,10292.0,8394
...,...,...,...,...,...,...
2020-02-27 11:45:00,2020-02-27 11:45:00,11325.0,11344.0,11315.0,11329.0,8389
2020-02-27 12:15:00,2020-02-27 12:15:00,11329.0,11329.0,11282.0,11283.0,11232
2020-02-27 12:45:00,2020-02-27 12:45:00,11282.0,11295.0,11246.0,11260.0,15699


In [34]:
# 準備 TXF 數據 feed
data_feed_txf = bt.feeds.PandasData(
    dataname=df,
    name='TXF',
    datetime=0,
    high=2,
    low=3,
    open=1,
    close=4,
    volume=5,
    plot=False
)


In [35]:
cerebro.adddata(data_feed_txf, name='TXF')
# 設定初始現金及手續費信息
cerebro.broker.setcash(600000.0)
cerebro.broker.setcommission(commission=200, margin=354000, mult=200, name='TXF')
# 添加策略制 cerebro
cerebro.addstrategy(SampleStrategy)
# 輸出初始組合資產價值
print(f'初始組合資產價值: {cerebro.broker.getvalue():.2f}')

初始組合資產價值: 600000.00


In [36]:
# 添加 PyFolio 分析器
cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')

# 執行回測
results = cerebro.run()

2019-03-04T00:00:00, 創建 0050 買入訂單, 大小: -25316
2019-03-05T00:00:00, 買入執行 0050,
                         價格: 15.73,
                         成本: 398175.98,
                         手續費: 398.18
2019-03-06T13:45:00, 0050 獲利回撤超過 10%, 創建 TXF 賣出訂單
2019-03-11T13:45:00, 0050 盈利回升, 平倉
2019-03-12T13:45:00, 0050 獲利回撤超過 10%, 創建 TXF 賣出訂單
2019-03-14T13:45:00, 0050 盈利回升, 平倉
2019-03-22T13:45:00, 0050 獲利回撤超過 10%, 創建 TXF 賣出訂單
2019-03-28T13:45:00, 0050 盈利回升, 平倉
2019-04-25T13:45:00, 0050 獲利回撤超過 10%, 創建 TXF 賣出訂單
2019-04-26T13:45:00, 0050 盈利回升, 平倉
2019-05-03T13:45:00, 0050 獲利回撤超過 10%, 創建 TXF 賣出訂單
2019-05-06T13:45:00, 0050 盈利回升, 平倉
2019-05-07T13:45:00, 0050 獲利回撤超過 10%, 創建 TXF 賣出訂單
2019-07-12T13:45:00, 0050 盈利回升, 平倉
2019-07-16T13:45:00, 0050 獲利回撤超過 10%, 創建 TXF 賣出訂單
2019-07-18T13:45:00, 0050 盈利回升, 平倉
2019-07-30T13:45:00, 0050 獲利回撤超過 10%, 創建 TXF 賣出訂單
2019-09-05T13:45:00, 0050 盈利回升, 平倉
2019-10-02T13:45:00, 0050 獲利回撤超過 10%, 創建 TXF 賣出訂單
2019-10-03T13:45:00, 0050 盈利回升, 平倉
2019-11-08T13:45:00, 0050 獲利回撤超過 10%, 創建 TXF